In [5]:
%pip install --user datasets
%pip install --user transformers
%pip install --user evaluate
%pip install --user torch
%pip install --user scikit-learn
%pip install --user pandas
%pip install --user matplotlib
%pip install --user seaborn
%pip install --user python-dotenv

Note: you may need to restart the kernel to use updated packages.


   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
    --------------------------------------- 0.3/12.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.0 MB 1.5 MB/s eta 0:00:08
   -- ------------------------------------- 0.8/12.0 MB 1.1 MB/s eta 0:00:11
   -- ------------------------------------- 0.8/12.0 MB 1.1 MB/s eta 0:00:11
   --- ------------------------------------ 1.0/12.0 MB 931.8 kB/s eta 0:00:12
   ---- ----------------------------------- 1.3/12.0 MB 958.5 kB/s eta 0:00:12
   ---- ----------------------------------- 1.3/12.0 MB 958.5 kB/s eta 0:00:12
   ----- ---------------------------------- 1.6/12.0 MB 902.1 kB/s eta 0:00:12
   ----- ---------------------------------- 1.6/12.0 MB 902.1 kB/s eta 0:00:12
   ------ --------------------------------- 1.8/12.0 MB 867.5 kB/s eta 0:00:12
   ------ -----

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install --upgrade protobuf
%pip install tf-keras

from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate, numpy as np, os

# -----------------------
# Configuration
# -----------------------
AFROXLMR_ID = "Davlan/afro-xlmr-base"       # change to large if you have resources
AFRIBERTA_ID = "castorini/afriberta_large"
NUM_LABELS = 3               # change if you have different number of sentiment classes
MAX_LEN = 128
BATCH = 8
EPOCHS = 10
OUTDIR = "checkpoints"
os.makedirs(OUTDIR, exist_ok=True)

Note: you may need to restart the kernel to use updated packages.


   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 258.3 kB/s eta 0:00:05
   ------------ ---------

In [ ]:
# -----------------------
# Load datasets
# -----------------------
train_path = "data/train.csv"
val_path   = "data/val.csv"
dataset = load_dataset("csv", data_files={"train": train_path, "validation": val_path})

# Optional: augment training set with lexicon-derived synthetic examples
lex_aug_path = "data/lexicon_augment.csv"   # columns: text,label  (optional)
if os.path.exists(lex_aug_path):
    aug_ds = load_dataset("csv", data_files={"train": lex_aug_path})["train"]
    # convert to HF Dataset compatible with same schema
    print("Adding lexicon-based augmentation:", len(aug_ds))
    dataset["train"] = concatenate_datasets([dataset["train"], aug_ds])


FileNotFoundError: Unable to find 'C:/Users/smadu/OneDrive/Desktop/INF 791 Applied Data Science/Assignment 03/This is your local repo/791-Group-Assignment\data/train.csv'

In [ ]:
# -----------------------
# Preprocessing helper
# -----------------------
def preprocess(tokenizer, ds):
    def tok(ex):
        return tokenizer(ex["text"], truncation=True, padding="max_length", max_length=MAX_LEN)
    tokenized = ds.map(tok, batched=True)
    # only keep fields needed for Trainer
    remove_cols = [c for c in tokenized["train"].column_names if c not in ("input_ids","attention_mask","label")]
    tokenized = tokenized.remove_columns(remove_cols)
    tokenized.set_format("torch")
    return tokenized


In [ ]:
# -----------------------
# Train function
# -----------------------
def train_model(model_id, save_dir):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=NUM_LABELS)

    tokenized = preprocess(tokenizer, dataset)

    metric_acc = evaluate.load("accuracy")
    metric_f1  = evaluate.load("f1")

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=-1)
        acc = metric_acc.compute(predictions=preds, references=labels)["accuracy"]
        f1 = metric_f1.compute(predictions=preds, references=labels, average="macro")["f1"]
        return {"accuracy": acc, "f1_macro": f1}

    training_args = TrainingArguments(
        output_dir=save_dir,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=BATCH,
        per_device_eval_batch_size=BATCH,
        num_train_epochs=EPOCHS,
        learning_rate=2e-5,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model="f1_macro",
        logging_steps=50
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized["train"],
        eval_dataset=tokenized["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    trainer.save_model(save_dir)
    tokenizer.save_pretrained(save_dir)
    print("Saved:", save_dir)


In [ ]:
# -----------------------
# Run training for both models
# -----------------------
if _name_ == "_main_":
    train_model(AFROXLMR_ID, os.path.join(OUTDIR, "afroxlmr"))
    train_model(AFRIBERTA_ID, os.path.join(OUTDIR, "afriberta"))